# Language Modeling 🤝 LSTM


In [1]:
import torch

from tqdm.auto import tqdm

## Подготовка данных

In [2]:
!wget -O arXiv.zip "https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1m78dRD6OIMP4oJL4VUujXV6MVavpb3A_"

!unzip arXiv.zip
!rm arXiv.zip

--2022-09-02 17:06:27--  https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1m78dRD6OIMP4oJL4VUujXV6MVavpb3A_
Resolving drive.google.com (drive.google.com)... 172.217.0.46, 2607:f8b0:4004:800::200e
Connecting to drive.google.com (drive.google.com)|172.217.0.46|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-0o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/uu375a1nk7rhf4m2nobd2mseou6kcu7e/1662138375000/06006207853926179639/*/1m78dRD6OIMP4oJL4VUujXV6MVavpb3A_?e=download&uuid=bc358784-b6d9-460b-8085-bad7f4979907 [following]
--2022-09-02 17:06:31--  https://doc-10-0o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/uu375a1nk7rhf4m2nobd2mseou6kcu7e/1662138375000/06006207853926179639/*/1m78dRD6OIMP4oJL4VUujXV6MVavpb3A_?e=download&uuid=bc358784-b6d9-460b-8085-bad7f4979907
Resolving doc-10-0o-docs.googleusercontent.com (doc-10-0o-docs.googleusercontent.com)... 142.251.111.132

In [3]:
import pandas as pd

arXiv_data = pd.read_json("arxivData.json")
arXiv_data.head()

,author,day,id,link,month,summary,tag,title,year
0,"[{'name': 'Ahmed Osman'}, {'name': 'Wojciech S...",1,1802.00209v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,We propose an architecture for VQA which utili...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Dual Recurrent Attention Units for Visual Ques...,2018
1,"[{'name': 'Ji Young Lee'}, {'name': 'Franck De...",12,1603.03827v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,Recent approaches based on artificial neural n...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Sequential Short-Text Classification with Recu...,2016
2,"[{'name': 'Iulian Vlad Serban'}, {'name': 'Tim...",2,1606.00776v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",6,We introduce the multiresolution recurrent neu...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Multiresolution Recurrent Neural Networks: An ...,2016
3,"[{'name': 'Sebastian Ruder'}, {'name': 'Joachi...",23,1705.08142v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,Multi-task learning is motivated by the observ...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Learning what to share between loosely related...,2017
4,"[{'name': 'Iulian V. Serban'}, {'name': 'Chinn...",7,1709.02349v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,We present MILABOT: a deep reinforcement learn...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",A Deep Reinforcement Learning Chatbot,2017


In [4]:
from random import choice

texts = arXiv_data["summary"].tolist()
print(choice(texts))

Determining the 3D structures of biological molecules is a key problem for
both biology and medicine. Electron Cryomicroscopy (Cryo-EM) is a promising
technique for structure estimation which relies heavily on computational
methods to reconstruct 3D structures from 2D images. This paper introduces the
challenging Cryo-EM density estimation problem as a novel application for
stochastic optimization techniques. Structure discovery is formulated as MAP
estimation in a probabilistic latent-variable model, resulting in an
optimization problem to which an array of seven stochastic optimization methods
are applied. The methods are tested on both real and synthetic data, with some
methods recovering reasonable structures in less than one epoch from a random
initialization. Complex quasi-Newton methods are found to converge more slowly
than simple gradient-based methods, but all stochastic methods are found to
converge to similar optima. This method represents a major improvement over
existing 

In [5]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from razdel import tokenize

SOS_TOKEN = '[SOS]'
EOS_TOKEN = '[EOS]'
PAD_TOKEN = '[PAD]'

vocabulary = set([SOS_TOKEN, EOS_TOKEN, PAD_TOKEN])
tokenized_texts = list()

for text in tqdm(texts[:1000]):
    # Токенизируем текст
    tokenized_text = tokenize(text.lower())
    tokenized_text = [token.text for token in tokenized_text]
    tokenized_text = [SOS_TOKEN] + tokenized_text + [EOS_TOKEN]

    # Обновим словарь
    for token in tokenized_text:
        vocabulary.add(token)

    # Добавим токенизированный текст в датасет
    tokenized_texts.append(tokenized_text)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [7]:
print(f"Vocabulary size is {len(vocabulary)}")
print(f"Tokenized example: {choice(tokenized_texts)}")

Vocabulary size is 9940
Tokenized example: ['[SOS]', 'in', 'this', 'paper', ',', 'we', 'propose', 'a', 'novel', 'unsupervised', 'domain', 'adaptation', 'algorithm', 'based', 'on', 'deep', 'learning', 'for', 'visual', 'object', 'recognition', '.', 'specifically', ',', 'we', 'design', 'a', 'new', 'model', 'called', 'deep', 'reconstruction-classification', 'network', '(', 'drcn', ')', ',', 'which', 'jointly', 'learns', 'a', 'shared', 'encoding', 'representation', 'for', 'two', 'tasks', ':', 'i', ')', 'supervised', 'classification', 'of', 'labeled', 'source', 'data', ',', 'and', 'ii', ')', 'unsupervised', 'reconstruction', 'of', 'unlabeled', 'target', 'data', '.', 'in', 'this', 'way', ',', 'the', 'learnt', 'representation', 'not', 'only', 'preserves', 'discriminability', ',', 'but', 'also', 'encodes', 'useful', 'information', 'from', 'the', 'target', 'domain', '.', 'our', 'new', 'drcn', 'model', 'can', 'be', 'optimized', 'by', 'using', 'backpropagation', 'similarly', 'as', 'the', 'standard

In [8]:
id_to_token = list(vocabulary)
token_to_id = {token: id for id, token in enumerate(id_to_token)}

Всё готово, осталось разделить выборку на обучающую и валидационную. Вторая нужна нам для промежуточного отслеживания качества:

In [9]:
from torch.utils.data import Dataset, DataLoader

class TextsForLM(Dataset):
    def __init__(self, texts):
        self.texts = list()

        for text in tqdm(texts):
            text_ids = [token_to_id[token] for token in text]

            self.texts.append(text_ids)

    def __getitem__(self, index):
        return self.texts[index]

    def __len__(self):
        return len(self.texts)

In [10]:
from sklearn.model_selection import train_test_split

train_texts, val_texts = train_test_split(tokenized_texts, test_size=0.1,
                                          random_state=42)

train_dataset = TextsForLM(train_texts)
val_dataset = TextsForLM(val_texts)

  0%|          | 0/900 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [11]:
def collate_texts(batch):
    max_length = 0
    for text_ids in batch:
        max_length = max(max_length, len(text_ids))

    for i in range(len(batch)):
        batch[i] += [token_to_id[PAD_TOKEN]] * (max_length - len(batch[i]))

    return torch.LongTensor(batch)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True,
                              collate_fn=collate_texts)

val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False,
                              collate_fn=collate_texts)

## Реализация модели


In [12]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 706 kB 22.6 MB/s 
     |████████████████████████████████| 5.9 MB 47.7 MB/s 
     |████████████████████████████████| 419 kB 51.4 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.0 which is incompatible.


In [13]:
from pytorch_lightning import LightningModule

In [14]:
from torch import nn, optim

class LMModel(LightningModule):
    def __init__(self, vocab_size, emb_dim=128, lstm_hidden_dim=128,
                 lstm_num_layers=2):
        super().__init__()

        self.embedding_layer = nn.Embedding(vocab_size, emb_dim)
        
        self.lstm = nn.LSTM(input_size=emb_dim, hidden_size=lstm_hidden_dim,
                            batch_first=True, num_layers=lstm_num_layers)

        self.output_layer = nn.Linear(lstm_hidden_dim, vocab_size)

    def forward(self, input_ids):
        # input_ids: [batch_size, seq_len]

        # embeddings: [batch_size, seq_len, emb_dim]
        embeddings = self.embedding_layer(input_ids)

        # context_repr: [batch_size, seq_len, lstm_hidden_dim]
        # h_n: [num_layers, batch_size, lstm_hidden_dim]
        # c_n: [num_layers, batch_size, lstm_hidden_dim]
        context_repr, (h_n, c_n) = self.lstm(embeddings)

        # logits: [batch_size, seq_len, vocab_size]
        logits = self.output_layer(context_repr)

        return logits, (h_n, c_n)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=3e-3)

        return optimizer

    def training_step(self, batch, _):
        #  input: SOS I      love   cats EOS
        # target: I.  love   cats.  EOS

        # pred:   [0.25, 0.25, 0.3, 0.0, 0.2]
        # target: [0.    0.    1.   0.   0. ]

        logits, (h_n, c_n) = self.forward(batch)

        batch_size, seq_len, vocab_size = logits.shape

        pred = logits[:, :-1, :].reshape(batch_size * (seq_len - 1), vocab_size)
        target = batch[:, 1:].reshape(batch_size * (seq_len - 1))

        loss_fn = nn.CrossEntropyLoss(ignore_index=token_to_id[PAD_TOKEN])

        loss = loss_fn(pred, target)

        return loss

In [15]:
lm_model = LMModel(len(vocabulary))

In [16]:
sample = next(iter(train_dataloader))

print(f"Sample shape is {sample.shape}")
print(f"Sample: {sample}")

Sample shape is torch.Size([16, 300])
Sample: tensor([[6485, 1918, 6821,  ..., 2923, 2923, 2923],
        [6485, 8377, 5467,  ..., 2923, 2923, 2923],
        [6485, 8160, 5897,  ..., 2923, 2923, 2923],
        ...,
        [6485, 2629,  166,  ..., 2923, 2923, 2923],
        [6485, 1511, 8211,  ..., 2923, 2923, 2923],
        [6485, 7904, 8948,  ..., 2923, 2923, 2923]])


## Обучение модели

Без лишних слов приступим к обучению модели

In [17]:
from pytorch_lightning import Trainer

trainer = Trainer(gpus=1, max_epochs=10)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [18]:
trainer.fit(lm_model, train_dataloader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type      | Params
----------------------------------------------
0 | embedding_layer | Embedding | 1.3 M 
1 | lstm            | LSTM      | 264 K 
2 | output_layer    | Linear    | 1.3 M 
----------------------------------------------
2.8 M     Trainable params
0         Non-trainable params
2.8 M     Total params
11.275    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


## Оценка качества

Нам определённо хочется понять, что мы только что наобучали


$$CrossEntropyLoss(y_1, \dots, y_n) = - \sum_{t=1}^{n} \log p(y_t | y_{<t})$$


$$Perplexity(y_1, \dots, y_n) = 2^{\frac{1}{n} CrossEntropyLoss(y_1, \dots, y_n)}$$


Наихудшим значением перплексии будет $V = vocab\_size$:

$$Perplexity(y_1, \dots, y_n) = 2^{\frac{1}{n} CrossEntropyLoss(y_1, \dots, y_n)} = 2^{- \frac{1}{n} \sum_{t=1}^{n} \log p(y_t | y_{<t})} = 2^{-\frac{1}{n} \cdot n \cdot \log \frac{1}{V}} = 2^{\log V} = V$$



In [19]:
def compute_perplexity(model, batch):
    values = []
    for token_ids in batch:
        logits, _ = model.forward(token_ids.unsqueeze(0))

        batch_size, seq_len, vocab_size = logits.shape

        pred = logits[0, :-1, :]
        target = token_ids[1:]

        loss_fn = nn.CrossEntropyLoss(ignore_index=token_to_id[PAD_TOKEN])

        loss = loss_fn(pred, target)

        values.append(2 ** loss.item())

    return torch.mean(torch.tensor(values))

In [20]:
print(f"Вспомним размер словаря: {len(vocabulary)}")

Вспомним размер словаря: 9940


In [21]:
untrained_lm_model = LMModel(len(vocabulary))

In [22]:
untrained_perplexities = []
trained_perplexities = []

for batch in tqdm(val_dataloader):
    untrained_perplexities.append(compute_perplexity(untrained_lm_model, batch))
    trained_perplexities.append(compute_perplexity(lm_model, batch))

print(f"Untrained Perplexity: {torch.mean(torch.tensor(untrained_perplexities))}")
print(f"Trained Perplexity: {torch.mean(torch.tensor(trained_perplexities))}")

  0%|          | 0/7 [00:00<?, ?it/s]

Untrained Perplexity: 587.65478515625
Trained Perplexity: 73.37569427490234


## Генерация



In [23]:
def choose_argmax(logits):
    """Выбирает наиболее вероятный токен"""

    next_token_id = logits[0, -1].argmax(dim=-1)

    return next_token_id

def sample_from_distribution(logits):
    """Строит распределение по логитам и семплирует из него"""

    dist = torch.distributions.categorical.Categorical(logits=logits[0, -1])
    next_token_id = dist.sample().item()

    return next_token_id

def sample_top_k_from_distribution(logits, k=40):
    """Выбирает k наиболее вероятных токенов и семплирует из них"""

    topv, topi = logits[0, -1].topk(k)

    dist = torch.distributions.categorical.Categorical(logits=topv)
    next_token_id = topi[dist.sample().item()].item()

    return next_token_id

def nucleus_sampling(logits, p=0.95):
    """Выбирает минимальный набор токенов, чья суммарная вероятность не меньше p,
       а затем семплирует из этого набораъ
       
       Подробнее: https://openreview.net/pdf?id=rygGQyrFvH
    """
    
    return None

def generate_sample(model, beginning, max_length,
                    sampling_strategy=sample_from_distribution,
                    temperature=1.0):
    
    if beginning is None:
        tokens = [token_to_id[SOS_TOKEN]]
    else:
        tokens = [token_to_id[token.text] for token in tokenize(beginning.lower())]
        tokens = [token_to_id[SOS_TOKEN]] + tokens

    for _ in range(max_length):
        tokens_tensor = torch.LongTensor(tokens).unsqueeze(0)

        logits, _ = model(tokens_tensor)
        logits /= temperature

        next_token_id = sampling_strategy(logits)

        if next_token_id == token_to_id[EOS_TOKEN]:
            break

        tokens.append(next_token_id)

    generated_sample = ' '.join([id_to_token[id] for id in tokens[1:]])

    return generated_sample

In [26]:
generate_sample(lm_model, "We propose", 20, sampling_strategy=sample_top_k_from_distribution)

'we propose the paper to their network for an and a novel neural , and used a networks with the deep data'